### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Prediction').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/15 22:22:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/15 22:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/15 22:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/15 22:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/15 22:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/04/15 22:22:52 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [3]:
## Read The dataset
import os
DATA_PATH = "data"
training = spark.read.csv(os.path.join(DATA_PATH,'test1.csv'),header=True,inferSchema=True)
training.show()

23/04/15 22:23:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [75]:
training.columns

['Name', 'age', 'Experience', 'Salary']

## Grop features as one new independent feature

In [ ]:
[Age,Experience]----> new feature--->independent feature

In [5]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [6]:
output=featureassembler.transform(training)

In [7]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [8]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

## Finalize the training dataframe

In [9]:
finalized_data=output.select("Independent Features","Salary")

In [10]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



## LinearRegression

In [11]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

23/04/15 22:35:56 WARN Instrumentation: [872951b4] regParam is zero, which might cause numerical instability and overfitting.
23/04/15 22:35:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/15 22:35:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/15 22:35:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
### Coefficients
regressor.coefficients

DenseVector([-79.3651, 1693.1217])

In [13]:
### Intercepts
regressor.intercept

15714.285714285747

In [14]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [15]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17275.132275132273|
|          [30.0,8.0]| 25000| 26878.30687830688|
+--------------------+------+------------------+



In [16]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1301.587301587304, 2026734.9738249276)